<a href="https://colab.research.google.com/github/DhrubojyotiDey/GenAI/blob/main/Genai_chat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install google-generativeai python-dotenv langchain PyPDF2 chromadb faiss-cpu langchain_google_genai -q

In [ ]:
from PyPDF2 import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os

from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.vectorstores import FAISS
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains.question_answering import load_qa_chain

from langchain.prompts import PromptTemplate

import google.generativeai as genai
from google.colab import files
import numpy as np
import json
import textwrap
import base64
import pathlib
import pprint
import requests
import mimetypes
from IPython.display import Markdown
from google.colab import userdata
from IPython.display import display , Markdown

In [ ]:
from langchain.chains import ConversationalRetrievalChain, RetrievalQA

In [ ]:
genai.configure(api_key=(userdata.get('Gapi'))) #authorization

In [ ]:


uploaded = files.upload()
for _ in uploaded.keys():
  path='/content/'+ _

In [ ]:
os.getcwd()

'/content'

In [ ]:
path = '/content/drive/MyDrive/alice.pdf'

In [ ]:
# Get the Google API key from userdata
google_api_key = userdata.get('Gapi')

# Set the environment variable
!export GOOGLE_API_KEY={google_api_key}

In [ ]:
#path = '/content/drive/MyDrive/alice.pdf'
def get_pdf_text(path):
    text = ""
    pdf_reader = PdfReader(path)
    for page in pdf_reader.pages:
        text += page.extract_text()
    return text

In [ ]:
get_pdf_text(path)

'\t\n\t\nAlice\'s\tAdventures\tin\tWonderland\n\t\nby\n\t\nLewis\tCarroll\n\t\nCHAPTER\tI.\tDown\tthe\nRabbit-Hole\n\t\nAlice\twas\tbeginning\tto\tget\tvery\ttired\tof\tsitting\tby\ther\tsister\ton\tthe\tbank,\tand\nof\thaving\tnothing\tto\tdo:\tonce\tor\ttwice\tshe\thad\tpeeped\tinto\tthe\tbook\ther\tsister\nwas\treading,\tbut\tit\thad\tno\tpictures\tor\tconversations\tin\tit,\t\'and\twhat\tis\tthe\tuse\nof\ta\tbook,\'\tthought\tAlice\t\'without\tpictures\tor\tconversations?\'\nSo\tshe\twas\tconsidering\tin\ther\town\tmind\t(as\twell\tas\tshe\tcould,\tfor\tthe\thot\nday\tmade\ther\tfeel\tvery\tsleepy\tand\tstupid),\twhether\tthe\tpleasure\tof\tmaking\ta\ndaisy-chain\twould\tbe\tworth\tthe\ttrouble\tof\tgetting\tup\tand\tpicking\tthe\tdaisies,\nwhen\tsuddenly\ta\tWhite\tRabbit\twith\tpink\teyes\tran\tclose\tby\ther.\nThere\twas\tnothing\tso\t\nvery\n\tremarkable\tin\tthat;\tnor\tdid\tAlice\tthink\tit\nso\t\nvery\n\tmuch\tout\tof\tthe\tway\tto\thear\tthe\tRabbit\tsay\tto\titself,\t\'Oh\

In [ ]:
def get_text_chunks(text):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=10000, chunk_overlap=1000)
    chunks = text_splitter.split_text(text)
    return chunks

In [ ]:
#embeddings = GoogleGenerativeAIEmbeddings(model = "models/embedding-001", google_api_key=userdata.get('Gapi'))
def get_vector_store(text_chunks):
    embeddings = GoogleGenerativeAIEmbeddings(model = "models/embedding-001", google_api_key=userdata.get('Gapi'))
    vector_store = FAISS.from_texts(text_chunks, embedding=embeddings)
    vector_store.save_local("faiss_index")

In [ ]:
vectr=get_vector_store(get_text_chunks(get_pdf_text('/content/drive/MyDrive/alice.pdf')))

In [ ]:
!ls faiss_index

ls: cannot access 'faiss_index': No such file or directory


In [ ]:
import shutil

shutil.rmtree('faiss_index')

In [ ]:
def get_qa_chain():

    prompt_template = """
    Given a pre-processed PDF document embedding context, answer the user's question Question in a way that simulates reading and understanding a storybook. Answers should be limited to 20 words per line.
    Context:\n {context}?\n
    Question: \n{question}\n

    Answer:
    """
    model = ChatGoogleGenerativeAI(model="gemini-pro",temperature=0.3, google_api_key=google_api_key)

    prompt = PromptTemplate(template = prompt_template, input_variables = ["context", "question"])
    chain = load_qa_chain(model, chain_type="stuff", prompt=prompt)

    return chain

In [ ]:

user_query = input("State your question: ")
print("You asked:", user_query)

query=get_pdf_text(path)
text_chunks=get_text_chunks(query)
get_vector_store(text_chunks)

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001", google_api_key=google_api_key)
new_db = FAISS.load_local("faiss_index", embeddings, allow_dangerous_deserialization=True)
docs = new_db.similarity_search(user_query)

print(docs)


chain = get_qa_chain()

response = chain(
  {"input_documents": docs, "question": user_query},
  return_only_outputs=True
)

#print(response)
#print("Reply: ", response["output_text"])

response["output_text"]

State your question: who is queen?
You asked: who is queen?
[Document(page_content='\'I\twant\ta\tclean\tcup,\'\tinterrupted\tthe\tHatter:\t\'let\'s\tall\tmove\tone\tplace\ton.\'\nHe\tmoved\ton\tas\the\tspoke,\tand\tthe\tDormouse\tfollowed\thim:\tthe\tMarch\tHare\nmoved\tinto\tthe\tDormouse\'s\tplace,\tand\tAlice\trather\tunwillingly\ttook\tthe\tplace\nof\tthe\tMarch\tHare.\tThe\tHatter\twas\tthe\tonly\tone\twho\tgot\tany\tadvantage\tfrom\nthe\tchange:\tand\tAlice\twas\ta\tgood\tdeal\tworse\toff\tthan\tbefore,\tas\tthe\tMarch\nHare\thad\tjust\tupset\tthe\tmilk-jug\tinto\this\tplate.\nAlice\tdid\tnot\twish\tto\toffend\tthe\tDormouse\tagain,\tso\tshe\tbegan\tvery\ncautiously:\t\'But\tI\tdon\'t\tunderstand.\tWhere\tdid\tthey\tdraw\tthe\ttreacle\tfrom?\'\n\'You\tcan\tdraw\twater\tout\tof\ta\twater-well,\'\tsaid\tthe\tHatter;\t\'so\tI\tshould\tthink\nyou\tcould\tdraw\ttreacle\tout\tof\ta\ttreacle-well—eh,\tstupid?\'\n\'But\tthey\twere\t\nin\n\tthe\twell,\'\tAlice\tsaid\tto\tthe\tDormouse,\t

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


'This context does not mention anything about the queen, so I cannot answer this question from the provided context.'

In [ ]:
res=(response["output_text"])
print (res)

Alice is the protagonist of the story.


In [ ]:
type(docs)
new_db.similarity_search('alice')

In [ ]:
import os
os.environ['GOOGLE_API_KEY'] = 'Gapi'

In [ ]:
from googleapiclient.discovery import build

In [ ]:
retriever = new_db.as_retriever()
docs = retriever.invoke({"question": user_query})

GoogleGenerativeAIError: Error embedding content: "Could not recognize the intended type of the `dict`. A `Content` should have a 'parts' key. A `Part` should have a 'inline_data' or a 'text' key. A `Blob` should have 'mime_type' and 'data' keys. Got keys: ['question']"

In [ ]:
user_query="find the author"
docs = new_db.similarity_search(user_query)


chain = get_qa_chain()

response = chain(
  {"input_documents": docs, "question": user_query},
  return_only_outputs=True
)

print(response)
print("Reply: ", response["output_text"])


{'output_text': 'This question cannot be answered from the given context because the author is not mentioned in the provided text.'}
Reply:  This question cannot be answered from the given context because the author is not mentioned in the provided text.


In [ ]:
def ask_and_answer(faiss_index, google_api_key):
  """Prompts the user for a question, retrieves relevant text chunks, and generates an answer.

  Args:
      faiss_index: The FAISS index containing pre-computed similarity searches.
      google_api_key: Your Google Generative AI API key.

  Returns:
      None
  """

  # Prompt for user question
  user_query = input("State your question: ")

  # Retrieve relevant text chunks using the FAISS index
  embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001", google_api_key=google_api_key)
  docs = faiss_index.similarity_search(user_query)  # Assuming similarity_search returns relevant documents

  # Prepare the prompt template (adjust as needed)
  prompt_template = PromptTemplate("""
  Answer the question as detailed as possible from the provided context, make sure to provide all the details, if the answer is not in
  provided context just say, "answer is not available in the context", don't provide the wrong answer\n\n
  Context:\n {context}?\n
  Question: \n{question}\n

  Answer:
  """)

  # You might already have a question answering chain defined elsewhere
  # Replace this section with your existing logic for invoking the chain
  # ... (your question answering chain logic using prompt_template, docs, and user_query)

  # Print the answer (assuming the chain provides the answer in a specific variable)
  # print(response)
  print("Reply: ", answer_variable)  # Replace with the actual variable holding the answer

if __name__ == "__main__":
  # Assuming faiss_index is loaded elsewhere (from previous code execution)
  ask_and_answer(faiss_index, google_api_key)

NameError: name 'faiss_index' is not defined

what does chapter one talk about?

In [ ]:
print(get_vector_store.response())

In [ ]:
embeddings=GoogleGenerativeAIEmbeddings(model="models/embedding-001")

def user_input():

    user_query = input("state your question: ")

    new_db=FAISS.load_local("faiss_index", embeddings)
    docs=new_db.similarity_search(user_query)

    chain= get_qa_chain()

    response= chain(
        {"input_docuents":docs, "question": user_query}
        , return_only_outputs=True)

    print(response)

    print("Reply: ", response["output_text"])


In [ ]:
text=input("your question please: ")
print (text)

In [ ]:
def find_pdf_text(pdf_docs):
    text=""
    for pdf in pdf_docs:
        pdf_reader=PdfReader(pdf)
        for page in pdf_reader.pages:
            text+=page.extract_text()
    return text

uploaded = files.upload()
for _ in uploaded.keys():
  path='/content/'+ _


In [ ]:
import PyPDF2

def to_markdown(text):
  text=text.replace('.', ' *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

def find_pdf_text(pdf_file):
    """Extracts text from a single PDF file and returns it as a string."""

    try:
        with open(pdf_file, 'rb') as f:
            pdf_reader = PyPDF2.PdfReader(f)
            text = ""
            for page in pdf_reader.pages:
                text += page.extract_text()
            return to_markdown(text) # Remove leading/trailing whitespace

    except Exception as e:
        print(f"An error occurred while processing the PDF: {e}")
        return "Error: Unable to extract text from PDF."


# Example usage (assuming you've uploaded the PDF to Colab):
uploaded = files.upload()
pdf_path = '/content/' + list(uploaded.keys())[0]  # Get the uploaded file path
extracted_text = find_pdf_text(pdf_path)

if extracted_text != "Error: Unable to extract text from PDF.":
  print("Extracted Text:")
  print(extracted_text)
else:
  print(extracted_text)  # Display the error message if extraction failed


In [ ]:
!pip install PyMuPDF
!pip install pillow
!wget https://pubs.usgs.gov/circ/1441/circ1441.pdf


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.8/15.8 MB 38.2 MB/s eta 0:00:00
--2024-05-16 05:54:02--  https://pubs.usgs.gov/circ/1441/circ1441.pdf
Resolving pubs.usgs.gov (pubs.usgs.gov)... 18.160.213.28, 18.160.213.2, 18.160.213.99, ...
Connecting to pubs.usgs.gov (pubs.usgs.gov)|18.160.213.28|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 44073902 (42M) [application/pdf]
Saving to: ‘circ1441.pdf’

circ1441.pdf        100%[===================>]  42.03M  1.05MB/s    in 52s     

2024-05-16 05:54:56 (827 KB/s) - ‘circ1441.pdf’ saved [44073902/44073902]



In [ ]:
import os
import pymupdf
import PIL
import google.generativeai as genai


In [ ]:
os.getcwd()

# Define the folder name for saving images
pics_folder = "pics"

# Create the pics folder if it doesn't exist
if not os.path.exists(pics_folder):
  os.makedirs(pics_folder)  # Create the folder

doc = pymupdf.open("circ1441.pdf")  # Open a document

for page_index in range(len(doc)):
  page = doc[page_index]  # Get the page
  image_list = page.get_images()

  # Print the number of images found on the page
  if image_list:
    print(f"Found {len(image_list)} images on page {page_index}")
  else:
    print("No images found on page", page_index)

  for image_index, img in enumerate(image_list, start=1):
    xref = img[0]  # Get the XREF of the image
    pix = pymupdf.Pixmap(doc, xref)  # Create a Pixmap

    if pix.n - pix.alpha > 3:  # CMYK: convert to RGB first
      pix = pymupdf.Pixmap(pymupdf.csRGB, pix)

    # Save the image with a filename format and path to pics folder
    filename = f"{pics_folder}/page_{page_index}-image_{image_index}.png"
    pix.save(filename)  # Save the image as png
    pix = None

In [ ]:
image1 = PIL.Image.open('/content/pics/page_0-image_1.png')
image2 = PIL.Image.open('/content/pics/page_0-image_10.png')
image3 = PIL.Image.open('/content/pics/page_0-image_11.png')


In [ ]:

model = genai.GenerativeModel("gemini-pro-vision",generation_config={"temperature":1})

response = model.generate_content(
[f"""Your task is to provide a summary of the given pictures. use details to describle the provided image and also use context wherever applicable or necessary


""",
image1,image2,image3])
print(response.text)

ERROR:grpc._plugin_wrapping:AuthMetadataPluginCallback "<google.auth.transport.grpc.AuthMetadataPlugin object at 0x7fcdf8151d50>" raised exception!
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/google/auth/compute_engine/credentials.py", line 128, in refresh
    self._retrieve_info(request)
  File "/usr/local/lib/python3.10/dist-packages/google/auth/compute_engine/credentials.py", line 101, in _retrieve_info
    info = _metadata.get_service_account_info(
  File "/usr/local/lib/python3.10/dist-packages/google/auth/compute_engine/_metadata.py", line 323, in get_service_account_info
    return get(request, path, params={"recursive": "true"})
  File "/usr/local/lib/python3.10/dist-packages/google/auth/compute_engine/_metadata.py", line 248, in get
    raise exceptions.TransportError(
google.auth.exceptions.TransportError: ("Failed to retrieve http://metadata.google.internal/computeMetadata/v1/instance/service-accounts/default/?recursive=true from the Go

RetryError: Deadline of 60.0s exceeded while calling target function, last exception: 503 Getting metadata from plugin failed with error: ("Failed to retrieve http://metadata.google.internal/computeMetadata/v1/instance/service-accounts/default/?recursive=true from the Google Compute Engine metadata service. Status: 404 Response:\nb''", <google.auth.transport.requests._Response object at 0x7fcdfaaa6fe0>)